In [2]:
import cx_Oracle
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import argparse
import os
import IPython
from IPython.display import Image, display
import time

In [3]:
import pascal_utils as pas

Let's see all the available schemas, and the tables that are available in each. 

```javascript
SCHEMAS = ['CMS_HGC_CORE_ATTRIBUTE',
           'CMS_HGC_CORE_COND', 
           'CMS_HGC_HGCAL_COND', 
           'CMS_HGC_CORE_CONSTRUCT', 
           'CMS_HGC_CORE_IOV_MGMNT', 
           'CMS_HGC_CORE_MANAGEMNT']
```
Since all start with `CMS_HGC`, let's remove that from all, so
```javascript
SCHEMAS = [
            'CORE_ATTRIBUTE',
           (parts and data types)
           'CORE_COND', 
            (user-generated data, like tests)
            'HGCAL_COND', 
            (user-generated data spicific to HGCAL, like tests)
           'CORE_CONSTRUCT', 
           (parts)
               'CORE_IOV_MGMNT', 
                   (interval of validity of user-generated data)
           'CORE_MANAGEMNT'
            (locations, institutions, etc.)
    ]
```

In [16]:
pas.execute_query(
"""
SELECT table_name, num_rows from all_tables WHERE OWNER='CMS_HGC_HGCAL_COND'
ORDER BY num_rows DESC
""")


COLUMN NAMES:
 ['TABLE_NAME', 'NUM_ROWS']
\FINISHED 'execute_query' in 3.3594 SECS


,TABLE_NAME,NUM_ROWS
0,HGC_SENSOR_DEFECT_CHKS,NaN
1,HGC_PQC_DIODE_IV,NaN
2,HGC_PQC_DIODE_CV,NaN
3,HGC_PQC_FET,NaN
4,HGCROC_DACB_CONVEYOR_TEST,NaN
5,HGCROC_INJECTION_SCAN,NaN
6,HGCROC_PEDESTAL_RUN,NaN
7,HGCROC_PHASE_SCAN,NaN
8,HGCROC_RAM_RETENTION,NaN
9,HGCROC_SAMPLING_SCAN,NaN


See the registered wafer that has serial number "100113":

In [17]:
pas.execute_query("select * from CMS_HGC_CORE_CONSTRUCT.PARTS where SERIAL_NUMBER='100113'")


COLUMN NAMES:
 ['PART_ID', 'KIND_OF_PART_ID', 'LOCATION_ID', 'MANUFACTURER_ID', 'IS_RECORD_DELETED', 'RECORD_INSERTION_TIME', 'RECORD_INSERTION_USER', 'BARCODE', 'SERIAL_NUMBER', 'VERSION', 'NAME_LABEL', 'INSTALLED_DATE', 'REMOVED_DATE', 'INSTALLED_BY_USER', 'REMOVED_BY_USER', 'EXTENSION_TABLE_NAME', 'COMMENT_DESCRIPTION', 'RECORD_LASTUPDATE_TIME', 'RECORD_LASTUPDATE_USER', 'PRODUCTION_DATE', 'BATCH_NUMBER']
\FINISHED 'execute_query' in 2.5503 SECS


,PART_ID,KIND_OF_PART_ID,LOCATION_ID,MANUFACTURER_ID,IS_RECORD_DELETED,RECORD_INSERTION_TIME,RECORD_INSERTION_USER,BARCODE,SERIAL_NUMBER,VERSION,...,INSTALLED_DATE,REMOVED_DATE,INSTALLED_BY_USER,REMOVED_BY_USER,EXTENSION_TABLE_NAME,COMMENT_DESCRIPTION,RECORD_LASTUPDATE_TIME,RECORD_LASTUPDATE_USER,PRODUCTION_DATE,BATCH_NUMBER
0,25990,11080,5780,1000,F,2022-08-10 21:08:33,Alexander Joseph Wade (awade),None,100113,None,...,None,None,None,None,None,300um LD Si Wafer 100113,None,Alexander Joseph Wade (awade),None,None


We see that it was uploaded by (`RECORD_LASTUPDATE_USER`) Alex Wade, and where it was tested (`LOCATION_ID`) is 5780.

Now, what is this location in words?

In [4]:
pas.execute_query(
"""
SELECT table_name, num_rows from all_tables WHERE OWNER='CMS_HGC_CORE_MANAGEMNT'
ORDER BY num_rows DESC
""")


COLUMN NAMES:
 ['TABLE_NAME', 'NUM_ROWS']

FINISHED 'execute_query' in 3.2142 SECS


,TABLE_NAME,NUM_ROWS
0,USERS,NaN
1,LOCATIONS_HST,NaN
2,CONDITIONS_DATA_AUDITLOG,3657.0
3,LOCATIONS,40.0
4,INSTITUTIONS,34.0
5,INSTITUTIONS_HST,18.0


In [5]:
pas.execute_query(
"""
SELECT * FROM CMS_HGC_CORE_MANAGEMNT.INSTITUTIONS
""")


COLUMN NAMES:
 ['INSTITUTION_ID', 'IS_RECORD_DELETED', 'INSTITUTE_CODE', 'RECORD_INSERTION_TIME', 'RECORD_INSERTION_USER', 'RECORD_LASTUPDATE_TIME', 'RECORD_LASTUPDATE_USER', 'NAME', 'COMMENT_DESCRIPTION', 'TOWN', 'COUNTRY']

FINISHED 'execute_query' in 2.8045 SECS


,INSTITUTION_ID,IS_RECORD_DELETED,INSTITUTE_CODE,RECORD_INSERTION_TIME,RECORD_INSERTION_USER,RECORD_LASTUPDATE_TIME,RECORD_LASTUPDATE_USER,NAME,COMMENT_DESCRIPTION,TOWN,COUNTRY
0,3140,T,0.0,2021-09-30 14:44:55,CMS_HGC_CORE_MANAGEMNT,2022-02-15 23:09:43,CMS_HGC_CORE_MANAGEMNT,Father Institution,None,None,None
1,3960,F,1.0,2021-10-12 02:26:37,Alethea Butler-Nalin,2021-10-12 03:29:39,Alethea Butler-Nalin,NTU/NCU,None,Taipei,TWN
2,5120,F,0.0,2022-08-24 18:05:54,Alexander Joseph Wade (awade),NaT,Alexander Joseph Wade (awade),CERN->NCU,None,None,None
3,5121,F,0.0,2022-08-24 18:05:57,Alexander Joseph Wade (awade),NaT,Alexander Joseph Wade (awade),CERN->TTU,None,None,None
4,5122,F,0.0,2022-08-24 18:06:06,Alexander Joseph Wade (awade),NaT,Alexander Joseph Wade (awade),NCU,None,None,None
5,1500,F,0.0,2017-12-07 23:03:43,CMS_HGC_PRTTYPE_HGCAL_WRITER,NaT,None,SiDet,None,None,None
6,1501,F,0.0,2017-12-07 23:03:43,CMS_HGC_PRTTYPE_HGCAL_WRITER,NaT,None,UCSB HEP,None,None,None
7,1502,F,0.0,2017-12-07 23:03:44,CMS_HGC_PRTTYPE_HGCAL_WRITER,NaT,None,UMN HEP,None,None,None
8,1503,F,0.0,2017-12-07 23:03:44,CMS_HGC_PRTTYPE_HGCAL_WRITER,NaT,None,CERN DSF,None,None,None
9,1504,F,0.0,2017-12-07 23:03:45,CMS_HGC_PRTTYPE_HGCAL_WRITER,NaT,None,HEPHY,None,None,None


So, the question 
"where was sensor with serial number 100113 tested?"
translates to the query


In [7]:
pas.execute_query("""
select * from CMS_HGC_CORE_CONSTRUCT.PARTS,
CMS_HGC_CORE_MANAGEMNT.INSTITUTIONS.LOCATION_ID
where SERIAL_NUMBER='100113' 
INNER JOIN CMS_HGC_CORE_CONSTRUCT.PARTS.RECORD_INSERTION_USER on 
CMS_HGC_CORE_CONSTRUCT.PARTS.RECORD_INSERTION_USER = FROM CMS_HGC_CORE_MANAGEMNT.INSTITUTIONS
""")

DatabaseError: ORA-00933: SQL command not properly ended